In [ ]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
%xmode Minimal

Data source: the NOAA Extended Reconstructed Sea Surface Temperature (ERSST) v5 product, a widely used and trusted gridded compilation of of historical data going back to 1854.

In [ ]:
url = 'http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc'
ds = xr.open_dataset(url, drop_variables=['time_bnds'])
ds = ds.sel(time=slice('1960', '2018')).load()
ds

In [ ]:
ds.sst[0].plot(vmin=-2, vmax=30)

In [ ]:
ds.sst.sel(lon=300, lat=50).plot()

In [ ]:
ds.time.dt.month

In [ ]:
ds

In [ ]:
gb = ds.sst.groupby(ds.time.dt.month)
gb = ds.sst.groupby('time.month')
gb

In [ ]:
for group_name, group_da in gb:
    # stop iterating after the first loop
    break 

group_da.time.dt


In [ ]:
(gb.mean(dim = "lat") - ds.sst.mean(dim = "lat")).plot()

In [ ]:
# this does the same thing as the previous cell
sst_mm = gb.mean(dim='time')
sst_mm

In [ ]:
sst_mm.sel(lon=300, lat=50).plot()

In [ ]:
sst_mm.mean(dim='lon').transpose().plot.contourf(levels=12, vmin=-2, vmax=30)

In [ ]:
sst_mm.mean(dim='lon').plot.contourf(levels=12, vmin=-2, vmax=30)

In [ ]:
def remove_time_mean(x):
    return x - x.mean(dim='time')

ds_anom = ds.groupby('time.month').apply(remove_time_mean)
ds_anom

In [ ]:
gb = ds.groupby('time.month')
ds_anom = gb - gb.mean(dim='time')
ds_anom

In [ ]:
ds_anom_resample = ds_anom.resample(time='5Y').mean(dim='time')
ds_anom_resample

In [ ]:
ds_anom.sst.sel(lon=300, lat=50).plot()
ds_anom_resample.sst.sel(lon=300, lat=50).plot(marker='o')

In [ ]:
(ds_anom_resample.sel(time='2015-01-01', method='nearest') -
 ds_anom_resample.sel(time='1965-01-01', method='nearest')).sst.plot()

In [ ]:
ds_anom_rolling = ds_anom.rolling(time=12, center=True).mean()
ds_anom_rolling

In [ ]:
ds_anom.sst.sel(lon=300, lat=50).plot(label='monthly anom')
ds_anom_resample.sst.sel(lon=300, lat=50).plot(marker='o', label='5 year resample')
ds_anom_rolling.sst.sel(lon=300, lat=50).plot(label='12 month rolling mean', color='k')
plt.legend()


In [ ]:
ds.coarsen(time=12).mean()

In [ ]:
ds_coarse = ds.coarsen(lon=4, lat=4, boundary='pad').mean()
ds_coarse.sst.isel(time=0).plot(vmin=2, vmax=30, figsize=(12, 5), edgecolor='k')